<a href="https://colab.research.google.com/github/davidgoins236/public/blob/main/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Query Weather Data for Series of Secified Coordinates
Source: https://dev.meteostat.net/
Method:
- Install meteostat & Other libraries
- Specify filepaths and time parameters
- Upload coordinates & Convert to DF
- Loop through coordinates, identify nearest weather station, incorporate data
- Combine to new dataframe and save output

In [ ]:
#!pip install meteostat

In [ ]:
#import meteostat
from meteostat import Stations, Daily, Point

In [ ]:
#import libs & set parms

import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta



#path of coordinates to find weather history for 
fpath = 'https://raw.githubusercontent.com/davidgoins236/public/main/coords.csv'
output_fpath = "C:\\Users\\david\\Desktop"


start_year = 2010
start_month = 1
start_day_of_month = 1
today = datetime.combine(date.today(),datetime.min.time())

#threshold for min data from weather station
days_threshold = 0.8


In [ ]:
locs = pd.read_csv(fpath)
locs.head()

,ID,LAT,LON
0,274,32.259330,-90.133830
1,330,35.594020,-80.860080
2,214,43.131670,-89.302830
3,418,41.108600,-81.195200
4,637,32.009372,-102.156238


In [ ]:
#set start to end times
start = datetime(start_year, start_month, start_day_of_month)
end = today - timedelta(days = 1)
days = end-start
days= days.days

In [ ]:
#create empty arrays
result_list=[]
no_result_list=[]

In [ ]:
#loop through locations for weather data
for i in locs.itertuples(): 

  #find 10 closest weather stations
  stations = Stations()
  stations = stations.nearby(i[2],i[3])
  stations_df = stations.fetch(10).reset_index()
  station_list = stations_df.id.to_list()

  #loop through stations for loc
  for j in station_list:

    #set counter for # of locs completed
    results = 0

    # find weather data
    data = Daily(j,start,end)
    data = data.fetch()
    data =data.drop(['wdir','wpgt','pres','tsun'],axis = 1)

    #check whether any data was returned
    if data.shape[0] > (days * days_threshold):

      #add loc id to the df
      data['ID']=i[1]
      data['CLOSEST_STATION_ID']= j

      #append df to results
      result_list.append(data)

      #break if results were found
      break

    else:
      results = results + 1

    # check for any locations with no results
    if results == 10:
      no_result_list.append(i[1])

In [ ]:
#all daily weather by loc
df = pd.concat(result_list)

#create output filename
output_fname = output_fpath +'\\weather'+ today.strftime('%m_%d_%y')+'.csv'

df.to_csv(output_fname)


Testing below

In [143]:
import plotly.express as px

test_loc = df[df['ID']==2]
test_loc = test_loc.sort_values(['time'],ascending = False).reset_index().head(364)

px.bar(test_loc,x = 'time',y='prcp',title = 'Location 2 Preciptiation by Day T365')